In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
%pip install -qq -U datasets transformers pyarrow
%pip install -qq --upgrade transformers ftfy accelerate regex tqdm
%pip install git+https://github.com/openai/CLIP.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
beatrix-jupyterlab 2023.814.150030 requires jupyter-server~=1.16, but you have jupyter-server 2.12.1 which is incompatible.
beatrix-jupyterlab 2023.814.150030 requires jupyterlab~=3.4, but you have jupyterlab 4.0.5 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompati

Let's import all the libraries we need

In [7]:
import clip
import os
import torch
import requests
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from PIL import Image
from datasets import load_dataset
from pathlib import Path
import os

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"


**Download the CLIP model to encode the image**

In [19]:
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)


100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 205MiB/s]


**Let's Write a method to encode the image using clip model**

In [20]:
def calc_image_emb(img, model, preprocess, device):
    """
    This method computes the clip embeddings for a given image, after preprocessing it according to the model
    """
    image = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
    return image_features
    


In [21]:

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
img_features = calc_image_emb(image, clip_model, clip_preprocess, device)
print(img_features)

tensor([[-1.0590e-01,  1.3782e-01, -2.9712e-01,  1.9852e-02, -6.4331e-02,
         -1.6895e-01, -1.3501e-01, -2.5692e-03,  4.7412e-01, -1.7517e-01,
          2.4463e-01, -3.7964e-01,  4.7821e-02, -1.4014e-01, -3.4106e-01,
         -1.2634e-01, -2.3193e-01, -2.9712e-01,  1.7786e-01,  4.8645e-02,
         -1.3076e+00, -3.1921e-02,  4.2139e-01, -3.3301e-01, -4.7516e-02,
          2.9785e-01,  2.3926e-01, -1.8481e-01,  1.5759e-01, -4.7882e-02,
         -7.7759e-02,  2.5903e-01, -7.2937e-02,  1.7737e-01, -5.7764e-01,
         -5.2757e-03,  2.7808e-01, -2.8784e-01,  1.9055e-01,  3.2715e-01,
         -1.0114e-01, -3.4692e-01,  7.4234e-03, -1.4685e-01, -1.8994e-01,
          2.5225e-04,  5.2002e-01,  1.5112e-01, -9.1370e-02,  1.7554e-01,
          1.6333e-01,  2.5220e-01,  1.1707e-01, -5.2734e-02,  2.2852e-01,
          1.8579e-01,  2.5903e-01,  6.0254e-01, -2.5977e-01, -1.6711e-01,
          4.2505e-01, -1.3953e-01, -6.3782e-02,  3.5596e-01, -7.4036e-02,
         -2.7002e-01,  2.2437e-01,  1.

In [34]:
cc = img_features.squeeze().tolist()
print(len(cc))
print(cc[0])

512
-0.10589599609375


In [11]:

#dataset = load_dataset("json", data_files="llava_instruct_150k.json")
dataset = load_dataset("liuhaotian/LLaVA-Instruct-150K", data_files='llava_instruct_150k.json')


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
eg = next(iter(dataset['train']))
print(eg)

**We need to download COCO2017 images, and compute the CLIP embeddings and store them first**

In [36]:
def prepare_coco_image_embeddings(img_path_list):
    """
    This method computes the CLIP image embeddings for all the images in COCO 2017 dataset
    """
    f_base = Path("/path/to/file.txt").stem
    embeddings_dict = {}
    for f_name in tqdm(img_path_list):
        img = Image.open(f_name)
        img_embd = calc_image_emb(image, clip_model, clip_preprocess, device)
        embeddings_dict[f_name] = img_embd.squeeze().tolist()
    return embeddings_dict

def get_absolute_paths(directory_path):
    absolute_paths = []

    # Check if the given path is a valid directory
    if os.path.isdir(directory_path):
        # Iterate over all files in the directory
        for root, _, files in os.walk(directory_path):
            for file in files:
                # Construct the absolute path for each file
                absolute_path = os.path.abspath(os.path.join(root, file))
                absolute_paths.append(absolute_path)
    return absolute_paths

def prepare_files_list(dataset_path):
    dirs = ['train2017','val2017','test2017']
    files_list = []
    for each_dir in tqdm(dirs):
        files_list.extend(get_absolute_paths(os.path.join(dataset_path, each_dir)))
    return files_list

In [12]:
dataset_path = '/kaggle/input/coco-2017-dataset/coco2017'
files_list = prepare_files_list(dataset_path)


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
files_list[0]

'/kaggle/input/coco-2017-dataset/coco2017/train2017/000000501175.jpg'

In [22]:
embeddings_dict = prepare_coco_image_embeddings(files_list)

  0%|          | 0/163957 [00:00<?, ?it/s]

In [24]:
import pickle
pkl_file_path = '/kaggle/working/coco_embeddings.pkl'
json_file_path = '/kaggle/working/coco_embeddings.json'
with open(pkl_file_path,'wb') as fh:
    pickle.dump(embeddings_dict, fh)


In [25]:
import json

def write_dict_to_json(data_dict, file_path):
    """
    Write a dictionary to a JSON file.

    Parameters:
    - dictionary: The dictionary to be written to the file.
    - file_path: The path to the JSON file.
    """
    with open(file_path, 'w') as json_file:
        json.dump(data_dict, json_file, indent=4)

In [35]:

embeddings_dict_list = {}
for key, value in tqdm(embeddings_dict.items()):
    embeddings_dict_list[key] =  value.squeeze().tolist()

  0%|          | 0/163957 [00:00<?, ?it/s]

In [37]:
write_dict_to_json(embeddings_dict_list, json_file_path)

**Now we need to include the embeddings to the original json file of LLaVa